In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains

from webdriver_manager.chrome import ChromeDriverManager
import seleniumrequests
import time

import requests
import json
from urllib.parse import urlencode

from snowflake.sqlalchemy import URL
from urllib.parse import quote 
import sqlalchemy as sa
import pandas as pd
import datetime
from pandas import ExcelWriter
from datetime import date
from datetime import datetime, timedelta
import pickle

In [3]:
driver = seleniumrequests.Chrome(ChromeDriverManager().install())
#driver = webdriver.Chrome(ChromeDriverManager().install())
time.sleep(round(4))
print(">>> Opening website")
driver.get("https://account.dji.com/login?appId=dji_ag2_kr&backUrl=https%3A%2F%2Fagms-us.dji.com%2F")
#driver.maximize_window()
time.sleep(round(4))



[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.58M/6.58M [00:05<00:00, 1.37MB/s]


>>> Opening website


In [4]:
# Logging into website
driver.find_element(By.NAME, "username").click()
driver.find_element(By.NAME, "username").send_keys("marlonjuarez1998@gmail.com")
time.sleep(round(1))
driver.find_element(By.XPATH, "//*[@type='password' ]").click()
driver.find_element(By.XPATH, "//*[@type='password' ]").send_keys("agro20campodrones")
time.sleep(1)
#click button login
driver.find_element(By.XPATH, "//*[@type='primary']").click()
time.sleep(10)
#user agreement
driver.find_element(By.XPATH, "//*[@type='button' and @class='el-button el-button--primary']").click()
print(">>> Login succesful")
time.sleep(5)


>>> Login succesful


In [5]:
cookies = {}
selenium_cookies = driver.get_cookies()
for cookie in selenium_cookies:
    cookies[cookie['name']] = cookie['value']
print(cookies)


{'_bl_uid': 'hvle6bOpkbd22vs80g3UqmhdLUy2', 'acw_tc': '2ff6179a16708060618807025eba89e985a8d602a9664b8c0923531c32', 'ag_token': 'US_GnU-KYpMBkcHjxQC8FsxltZUd3Y-ZOAep3lCjglgZPrD9k0uavhmUArmsDcuQYGg11SZqD5SVYM7_iBUNaAhvyxk0mHSmewkNuFLUOeFdN2odYu-keMyjENclhZJ6D5UVWI1kEqC8FjvNcinFXxUnw', 'fb_v1': 'cr492kJDBSG_IDqJNBoGcYh-q8fre-bqPyk-sNu37MA', 'ga_uid': 'zypLWjEYMLnqNxc3GCF1OUTrupZMn3Togog-pFldfPM', '_ga': 'GA1.2.11562930.1670806057', '_logged': 'yes', '_meta_key': 'US_5Lo-j8dHN88G7bUEKg99-luWicyabHuUpa3aFR0Ue7nKwPIV-pJTNV8ZTcEto4Vh2Iu9gi8t6AGmwW4RXit-k4Yrk0V9PtUGzefqULZbyH4', '_gid': 'GA1.2.713988631.1670806057', 'dji_consentmanager': '%7B%22version%22%3A1%2C%22opt%22%3A%22accept%22%7D'}


In [6]:
n = 1
no_page = False
url = "https://agms-us.dji.com/proxy/ag/api/web/v1/flight_records?"

list_of_df = []

while no_page == False:
    
    print(n)
    payload={
        'page': n,
        'page_size': '50',
        'keyword': '',
        'filters[province_eq]': '',
        'filters[city_eq]': '',
        'filters[product_sn_eq]': '',
        'filters[create_date_gteq]': '1670306400',
        'filters[create_date_lteq]': '1670824799',
        'filters[new_work_area_gt]': '5'}

    try:
            response = requests.request("GET", url, cookies= cookies,params=payload)
            #print(response.text)

            data = response.json()
            data = data['data']
            
            if len(data) == 0:
                
                break

            for d in data:
                list_of_df.append(d)  
            
            n+=1            
    except:
            print('No hay pagina')
            no_page = True
    
print(">>> Data requested correctly")  

1
2
3
4
>>> Data requested correctly


In [7]:
driver.close()

In [8]:
concated_df = pd.DataFrame(list_of_df)

In [9]:
concated_df['start_timestamp'] = pd.to_datetime(concated_df['start_timestamp'] ,unit='s', utc=True)
concated_df['end_timestamp'] = pd.to_datetime(concated_df['end_timestamp'],unit='s', utc=True)
concated_df['created_at'] = pd.to_datetime(concated_df['created_at'],unit='s', utc=True)

concated_df['start_timestamp'] = concated_df.start_timestamp.dt.tz_convert('America/Guatemala')
concated_df['end_timestamp'] = concated_df.end_timestamp.dt.tz_convert('America/Guatemala')
concated_df['created_at'] = concated_df.created_at.dt.tz_convert('America/Guatemala')


concated_df['start_timestamp'] = concated_df['start_timestamp'].dt.tz_localize(None)
concated_df['end_timestamp'] = concated_df['end_timestamp'].dt.tz_localize(None)
concated_df['created_at'] = concated_df['created_at'].dt.tz_localize(None)
concated_df

,id,flyer_name,location,city,district,create_date,team_name,no,task_serial_number,manual_mode,...,end_timestamp,work_time_seconds,created_at,updated_at,mode_name,work_speed,spray_width,radar_height,plot_name,mission_serial_number
0,66361309,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",None,None,20221210,APLICACION GENERALES,None,None,True,...,2022-12-10 09:38:46,352,2022-12-10 14:25:47,1670703947,1,NaN,NaN,NaN,None,None
1,66361308,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",None,None,20221210,APLICACION GENERALES,None,None,False,...,2022-12-10 09:32:54,256,2022-12-10 14:25:47,1670703947,4,7.000,8.18,7.5,None,None
2,66361307,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",None,None,20221210,APLICACION GENERALES,None,None,False,...,2022-12-10 09:23:03,541,2022-12-10 14:25:46,1670703946,4,7.000,8.18,7.5,None,None
3,66353595,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",None,None,20221210,APLICACION GENERALES,None,None,False,...,2022-12-10 09:09:16,612,2022-12-10 09:12:11,1670685131,4,7.000,8.18,7.5,None,None
4,66352590,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",None,None,20221210,APLICACION GENERALES,None,None,True,...,2022-12-10 07:59:03,76,2022-12-10 08:57:29,1670684249,1,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,65756328,Marlon Juárez,"Cuyotenango, Suchitepéquez, Guatemala",None,None,20221206,APLICACION GENERALES,None,None,False,...,2022-12-06 06:38:55,615,2022-12-06 12:03:28,1670349808,4,6.808,7.02,8.5,None,None
112,65756429,Marlon Juárez,"Cuyotenango, Suchitepéquez, Guatemala",None,None,20221206,APLICACION GENERALES,None,None,False,...,2022-12-06 06:25:46,493,2022-12-06 12:10:37,1670350237,4,6.700,7.04,8.5,None,None
113,65756327,Marlon Juárez,"Cuyotenango, Suchitepéquez, Guatemala",None,None,20221206,APLICACION GENERALES,None,None,False,...,2022-12-06 06:25:47,615,2022-12-06 12:03:28,1670349808,4,6.808,7.02,8.5,None,None
114,65756428,Marlon Juárez,"Cuyotenango, Suchitepéquez, Guatemala",None,None,20221206,APLICACION GENERALES,None,None,False,...,2022-12-06 06:12:38,516,2022-12-06 12:10:36,1670350236,4,6.904,7.04,8.5,None,None


In [10]:
concated_df = concated_df[['flyer_name','location','team_name','spray_usage','nickname',
                           'new_work_area','start_timestamp','end_timestamp','work_time_seconds',
                           'mode_name','work_speed','spray_width','radar_height','plot_name','mission_serial_number']]


concated_df.dtypes

flyer_name                       object
location                         object
team_name                        object
spray_usage                       int64
nickname                         object
new_work_area                     int64
start_timestamp          datetime64[ns]
end_timestamp            datetime64[ns]
work_time_seconds                 int64
mode_name                         int64
work_speed                      float64
spray_width                     float64
radar_height                    float64
plot_name                        object
mission_serial_number            object
dtype: object

In [11]:
concated_df

,flyer_name,location,team_name,spray_usage,nickname,new_work_area,start_timestamp,end_timestamp,work_time_seconds,mode_name,work_speed,spray_width,radar_height,plot_name,mission_serial_number
0,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",APLICACION GENERALES,7731,Viserys,5566,2022-12-10 09:32:54,2022-12-10 09:38:46,352,1,NaN,NaN,NaN,None,None
1,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",APLICACION GENERALES,5207,Viserys,3473,2022-12-10 09:28:38,2022-12-10 09:32:54,256,4,7.000,8.18,7.5,None,None
2,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",APLICACION GENERALES,20695,Viserys,13826,2022-12-10 09:14:02,2022-12-10 09:23:03,541,4,7.000,8.18,7.5,None,None
3,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",APLICACION GENERALES,28085,Viserys,18793,2022-12-10 08:59:04,2022-12-10 09:09:16,612,4,7.000,8.18,7.5,None,None
4,Marlon Juárez,"Nueva Concepción, Escuintla, Guatemala",APLICACION GENERALES,2968,Viserys,3500,2022-12-10 07:57:46,2022-12-10 07:59:03,76,1,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Marlon Juárez,"Cuyotenango, Suchitepéquez, Guatemala",APLICACION GENERALES,11637,Viserys,7526,2022-12-06 06:28:40,2022-12-06 06:38:55,615,4,6.808,7.02,8.5,None,None
112,Marlon Juárez,"Cuyotenango, Suchitepéquez, Guatemala",APLICACION GENERALES,27383,Vaghar,16020,2022-12-06 06:17:33,2022-12-06 06:25:46,493,4,6.700,7.04,8.5,None,None
113,Marlon Juárez,"Cuyotenango, Suchitepéquez, Guatemala",APLICACION GENERALES,22332,Viserys,15046,2022-12-06 06:15:32,2022-12-06 06:25:47,615,4,6.808,7.02,8.5,None,None
114,Marlon Juárez,"Cuyotenango, Suchitepéquez, Guatemala",APLICACION GENERALES,20320,Vaghar,14226,2022-12-06 06:04:02,2022-12-06 06:12:38,516,4,6.904,7.04,8.5,None,None


In [12]:
concated_df.to_excel('dron.xlsx', sheet_name='hoja')

In [13]:
n=1
s = requests.Session() 
url = "https://agrodronesdelcampo.000webhostapp.com/data_dron.php?"

for index, row in concated_df.iterrows():
    
    payload={
        'flyer': row['flyer_name'],
        'location': row['location'],
        'team': row['team_name'],
        'spu': row['spray_usage'],
        'nick': row['nickname'],
        'nwa': row['new_work_area'],
        'start': row['start_timestamp'],
        'end': row['end_timestamp'],
        'wkts': row['work_time_seconds'],
        'mode': row['mode_name'],
        'wksp': row['work_speed'],
        'spw': row['spray_width'],
        'rh': row['radar_height'],
        'plot': row['plot_name'],
        'mission': row['mission_serial_number']}


    response = s.post(url, data=payload)    
    time.sleep(1)
    
    if n%200 == 0:
        print(index)
        time.sleep(60)
    n = n+1
print("DATA MIGRATION COMPLETED")        

DATA MIGRATION COMPLETED
